In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [2]:
def splitDateInColumns(df):
    for index, row in df.iterrows():
        date_object = datetime.strptime(row["Date"], '%d.%m.%Y')
        df.at[index, 'Day'] = date_object.day
        df.at[index, 'Month'] = date_object.month
        df.at[index, 'Year'] = date_object.year
    return df

In [8]:
df = pd.read_csv("/Users/BramSikkens/Desktop/AI-FinalWork/data.csv",sep=";",error_bad_lines=False,)

OSError: Initializing from file failed

In [4]:
def cleanColumns(df):
    df = df.dropna(subset=['Date', 'Person 1', 'Person 2',
                           'split time 1', 'split time 2', 'split time 3', 'Total Time'])
    return df

In [5]:

# Solution from : https://stackoverflow.com/questions/32854677/how-to-deal-with-multiple-date-string-formats-in-a-python-series
def timeToSeconds(timestr):
    for fmt in ("%M:%S.%f", "%S.%f", "%S:%f", "%M:%S:%f"):
        try:
            t = datetime.strptime(timestr, fmt)
            seconds = (t.minute * 60) + t.second + (t.microsecond * 1e-6)
            return round(seconds)
            break
        except ValueError:
            pass

In [6]:
def convertTimes(df):
    for index, row in df.iterrows():
        df.at[index, "split time 1"] = timeToSeconds(row["split time 1"])
        df.at[index, "split time 2"] = timeToSeconds(row["split time 2"])
        df.at[index, "split time 3"] = timeToSeconds(row["split time 3"])
        df.at[index, "Total Time"] = timeToSeconds(row["Total Time"])


In [7]:
def createCompetitionColumns(df):
    df = pd.get_dummies(df,columns=['Competiton Type','Competition Round'], dtype=bool)
    return df

In [8]:
def create_DeltaTimes(df):
     for index, row in df.iterrows():
        df.at[index,"D0_250"] = row["split time 1"]
        df.at[index,"D250_500"] = pd.to_numeric(row["split time 2"])-pd.to_numeric(row["split time 1"])
        df.at[index,"D500_750"] = pd.to_numeric(row["split time 3"])-pd.to_numeric(row["split time 2"])
        df.at[index,"D750_1000"] = pd.to_numeric(row["Total Time"])-pd.to_numeric(row["split time 3"])




In [ ]:
def create_tactic_columns:
    #Fast/Neutral/Slow Start?
    #Fast/Neutral/Slow  Middle?
    #Fast/Neutral/Slow End? 

In [ ]:
def createRaceIndex():


In [ ]:
def createPb_TimecurrentSeason():


In [ ]:
def create_Pb_Rank_currentSeason():

In [ ]:
def create_Reached_FinalA():
    

In [ ]:
def create_AVG_Rank_currentSeason():
    

In [ ]:
def create_Reached_Semi():

In [ ]:
def create_FinalRank():

In [ ]:
def create_count_FinalA_currentSeason():
    

In [ ]:
def create_FinalARatio_currentSeason():
    